In [1]:
import string

In [2]:
from IPython.display import Image, clear_output, display
from Libs.Beholder import *
from PIL import Image as pilImage

a = Beholder(videoFrameGenerator=pullPhoneScreen)

a.addLayerModifer(
    Beholder_Layer_Chopper_Grayscale(name="gray", from_layer="image", enabled=True)
)

a.addLayerModifer(
    Beholder_Layer_Chopper_AtCord(
        name="gray_letters",
        from_layer="gray",
        x=304,
        y=1632,
        w=850,
        h=860,
        enabled=True,
    )
)

In [7]:
a.matchers = {}
for l in string.ascii_uppercase:
    p = Path(f"./wordtrip_templates/{l}.png")
    if p.exists():
        print(l)
        a.addMatcher(
            Beholder_Image_Matcher(
                name=l, layer="gray_letters", filename=str(p), threshhold=0.2
            )
        )

B
C
D
L
O
S
T
U


In [4]:
from pathlib import Path

import requests

w = Path("words.lst")
if not w.exists():
    w.write_bytes(
        requests.get(
            "https://raw.githubusercontent.com/MariaLuo826/english_words/master/words_alpha.txt"
        ).content
    )
words = [l.upper() for l in w.read_text().splitlines()]
len(words)

370105

In [8]:
def wordWorks(w, found_letters):
    for l in w:
        if l in found_letters:
            found_letters.remove(l)
        else:
            return False
    return True

In [11]:
min_len = 3
eq_len = 3
matches = a.findMatches()
print(matches.keys())
for w in words:
    found_letters = list(matches.keys())
    if eq_len is not None:
        if eq_len != len(w):
            continue
    if len(w) < min_len:
        continue
    if len(w) > len(found_letters):
        continue
    if wordWorks(w, found_letters):
        print(w)

dict_keys(['B', 'D', 'O', 'T', 'U'])
BOD
BOT
BTU
BUD
BUT
DOB
DOT
DUB
DUO
OUD
OUT
TOB
TOD
TOU
TUB
UDO
